In Context evaluation of the best performing models.

We now wish to evaluate each of our projection models at the token level. We construct an evaluation dataset from a set of homonyms from the McRae feature set. The dataset consists of tokens of homonymous words collected from semcor, alongside gold feature data (mcrae) for disambiguated senses.

We have gold features in mcrae and buchanan format, so we only use these datasets, not binder.


We have trained models with best performing hyperparameters on the other words in the datasets.

4 layer ffnn (1k)
4 layer ffnn (5k)
4 layer ffnn (glove)
plsr (1k)
plsr (5k)
plsr(glove)
label propagation (1k)
label propagation (5k)
label propagation (glove)

To evaluate a model, we use it to predict features in context for each token in the dataset. Then, we compare to the gold feature data. We also calculate the average cosine distance between gold and predicted vectors. 

In [1]:
import sys
sys.path.append("../src/")
sys.path.append("../")

import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt


from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

from scipy.stats import spearmanr

bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmp9vmr7789
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

We want to evaluate the model on its ability to predict features in context. First, we obtain gold vectors for both senses of 10 ambiguous words from the McRae et al. data

What are the words?

# Construct the Evaluation Dataset.

In [6]:
save_path = '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters'
model = torch.load(save_path)

df = pd.read_csv('../data/processed/bnc_contexts_for_mcrae_homonyms.csv')
feature_norms = BuchananFeatureNorms('../data/external/buchanan/cue_feature_words.csv')



evaluate_in_context(model, df, bert, feature_norms)

NameError: name 'evaluate_in_context' is not defined

In [7]:
ambiguous_pairs = [
    ('bat_animal', 'bat_baseball'),
    ('board_wood', 'board_black'),
    ('bow_ribbon', 'bow_weapon'),
    ('cap_bottle', 'cap_hat'),
    #('crane_machine', 'crane_animal')
    ('hose', 'hose_leggings'),
    ('mink', 'mink_coat'), # this one is not fully disambiguated
    ('mouse', 'mouse_computer'),
    ('pipe_smoking', 'pipe_plumbing'),
    ('tank_army', 'tank_container')
]

In [8]:
[item for t in ambiguous_pairs for item in t]


['bat_animal',
 'bat_baseball',
 'board_wood',
 'board_black',
 'bow_ribbon',
 'bow_weapon',
 'cap_bottle',
 'cap_hat',
 'hose',
 'hose_leggings',
 'mink',
 'mink_coat',
 'mouse',
 'mouse_computer',
 'pipe_smoking',
 'pipe_plumbing',
 'tank_army',
 'tank_container']

In [17]:
"""
take a look at the gold features for these words.
"""
norms = BuchananFeatureNorms('../data/external/buchanan/cue_feature_words.csv')


norms.print_features('pipe')
norms.print_features('pipe_plumbing')
norms.print_features('tank_container')

features for pipe: {'cylinder': 20.0, 'metal': 20.0, 'plumbing': 16.66666667, 'smoke': 40.0, 'tube': 23.33333333, 'water': 23.33333333}
features for pipe_plumbing: {'carry': 26.66666667, 'connect': 16.66666667, 'copper': 23.33333333, 'hard': 23.33333333, 'hold': 26.66666667, 'hollow': 40.0, 'leak': 20.0, 'liquid': 20.0, 'long': 43.33333333, 'metal': 60.0, 'plastic': 20.0, 'round': 56.66666667, 'transport': 20.0, 'water': 53.33333333}
features for tank_container: {'fish': 26.66666667, 'glass': 20.0, 'hold': 80.0, 'keep': 26.66666667, 'large': 26.66666667, 'liquid': 30.0, 'metal': 36.66666667, 'plastic': 33.33333333, 'size': 20.0, 'store': 23.33333333, 'thing': 56.66666667, 'water': 23.33333333}


Now we want to find examples of each sense and make predictions using the model for that word. We will score the predictions of each example and then average them. Here are 10 examples of bat_animal. Load these from the file we have created with these words.

# Train the Models

Hopefully this is done already 

In [2]:
"""
models and save paths

"""
buchanan_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters',
    '../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.2dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/main_d62d4_00011_11_clusters=5,embedding_type=bert,model=modabs,mu2=1e-07,mu3=0.001,mu4=20,nnk=6,train_data=mc_rae_real_2022-10-15_18-12-18', # mcrae is just a misnomer this was trained on buchanan, or else it wouldnt work
    '../trained_models/main_094eb_00082_82_clusters=1,embedding_type=bert,model=modabs,mu2=0.1,mu3=10.0,mu4=20,nnk=6,train_data=buchanan_2022-10-12_05-39-12',
]


buchanan_glove_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.glove.50epochs.0.0dropout.lr1e-4.hsize300',
    '../trained_models/main_094eb_00035_35_clusters=1,embedding_type=glove,model=modabs,mu2=1e-07,mu3=1.0,mu4=5,nnk=6,train_data=buchanan_2022-10-12_02-26-58'   
]

mcrae_models = [
     '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
     '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
     '../trained_models/model.plsr.mc_rae_real.allbuthomonyms.5k.100components.500max_iters',
     '../trained_models/main_0a52f_00012_12_clusters=1,embedding_type=bert,model=plsr,plsr_max_iter=500,plsr_n_components=100,train_data=mc_rae_real_2022-10-11_22-50-43',
     '../trained_models/main_d62d4_00011_11_clusters=5,embedding_type=bert,model=modabs,mu2=1e-07,mu3=0.001,mu4=20,nnk=6,train_data=mc_rae_real_2022-10-15_18-12-19',
     '../trained_models/main_094eb_00128_128_clusters=1,embedding_type=bert,model=modabs,mu2=0.001,mu3=0.001,mu4=5,nnk=6,train_data=mc_rae_real_2022-10-12_08-37-21',
]

mcrae_glove_models = [
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/main_0a52f_00013_13_clusters=1,embedding_type=glove,model=plsr,plsr_max_iter=500,plsr_n_components=100,train_data=mc_rae_real_2022-10-11_22-50-43',
    '../trained_models/main_094eb_00027_27_clusters=1,embedding_type=glove,model=modabs,mu2=0.001,mu3=0.001,mu4=1,nnk=4,train_data=mc_rae_real_2022-10-12_02-23-04',
]

# this analysis is only for buchanan and mcrae
# binder_models = [
#     '../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300',
#     '../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
#     '../trained_models/model.plsr.binder.5k.300components.500max_iters',
#     '../trained_models/model.plsr.binder.1k.300components.500max_iters',
#     '../trained_models/model.modabs.binder.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
#     '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
# ]

# binder_glove_models = [
#     '../trained_models/model.ffnn.binder.glove.50epochs.0.5dropout.lr1e-4.hsize300',
#     '../trained_models/model.plsr.binder.glove.300components.500max_iters',
#     '../trained_models/model.modabs.binder.glove.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',    
# ]

# Run the models on the data

TODO are you sure this is the exact dataset you want? Per Katrins comment:

Another small question on the McRae homonym analysis: If there was a word labeled with sense cap.n.02 but it's not "cap" but another word, say "lid", do you also use that in this analysis, or only the correct lemma?

In [10]:
for model in buchanan_models:
    torch.load(model)

In [9]:
"""
10_15_2022 now you are using the much larger expanded dataset
"""

pd.set_option('display.max_colwidth', None)
eval_words = pd.read_csv('../data/processed/mcrae_homonyms_from_coca_10_17_2022.csv')
print(eval_words.head(20))


    id word                                   source  \
0    1  bat      COCA:2019:NEWS\nDallas Morning News   
1    2  bat                 COCA:1991:FIC\nKenyonRev   
2    3  bat                   COCA:2000:NEWS\nDenver   
3    4  bat               COCA:1995:FIC\nMov:BadBoys   
4    5  bat   COCA:2012:WEB\n...logs.mercurynews.com   
5    6  bat                COCA:1999:FIC\nBk:Riptide   
6    7  bat                 COCA:2018:FIC\nNewYorker   
7    8  bat               COCA:2005:FIC\nLiteraryRev   
8    9  bat             COCA:2006:MOV\nDirty Laundry   
9   10  bat      COCA:1990:TV\nMarried with Children   
10  11  bat              COCA:2004:MAG\nConsumResrch   
11  12  bat            COCA:1997:SPOK\nABC_Nightline   
12  13  bat                COCA:2003:MAG\nMensHealth   
13  14  bat                  COCA:2003:NEWS\nHouston   
14  15  bat          COCA:2016:MAG\nBusiness Insider   
15  16  bat      COCA:2019:NEWS\nDallas Morning News   
16  17  bat                   COCA:2001:MAG\nEsq

In [10]:
# prune to be no more than 50 tokens per word (the least we have is like 16 for crane (bird))
eval_words = eval_words.groupby('buchanan_cue_word').apply(lambda s: s.sample(min(len(s), 50)))
print(len(eval_words))
print(eval_words.groupby(['mcrae_cue_word']).count())


794
                  id  word  source  sentence  buchanan_cue_word
mcrae_cue_word                                                 
bat_(animal)      50    50      50        50                 50
bat_(baseball)    50    50      50        50                 50
board_(black)     28    28      28        28                 28
board_(wood)      50    50      50        50                 50
bow_(ribbon)      43    43      43        43                 43
bow_(weapon)      50    50      50        50                 50
cap_(bottle)      20    20      20        20                 20
cap_(hat)         50    50      50        50                 50
crane_(animal)    14    14      14        14                 14
crane_(machine)   50    50      50        50                 50
hose              42    42      42        42                 42
hose_(leggings)   50    50      50        50                 50
mink              32    32      32        32                 32
mink_(coat)       33    33      33  

In [13]:
print(eval_words.groupby(['mcrae_cue_word']).count().mean(axis=0))


id                   39.7
word                 39.7
source               39.7
sentence             39.7
buchanan_cue_word    39.7
dtype: float64


In [52]:
eval_words.head(20)

id word                source                \
buchanan_cue_word                                                    
bat_animal        92   42  bat         COCA:2004:ACAD\nNaturalHist   
                  57    7  bat   COCA:2013:FIC\nBk:TwoSerpentsRise   
                  95   45  bat         COCA:2003:ACAD\nNaturalHist   
                  102  52  bat     COCA:2014:ACAD\nJAnimalPlantSci   
                  72   22  bat          COCA:2008:ACAD\nBioscience   
                  78   28  bat     COCA:2014:ACAD\nJAnimalPlantSci   
                  62   12  bat    COCA:2008:FIC\nBk:DeathDeceitAmp   
                  67   17  bat         COCA:1990:FIC\nTriquarterly   
                  74   24  bat          COCA:2008:ACAD\nBioscience   
                  97   47  bat         COCA:2003:ACAD\nNaturalHist   
                  60   10  bat  COCA:2012:WEB\nplanet.infowars.com   
                  91   41  bat          COCA:2008:ACAD\nBioscience   
                  71   21  bat          COCA:2008:ACAD\nBioscience   
                  68   18  bat            COCA:1990:FIC\nBk:Malibu   
                  52    2  bat             COCA:2019:TV\nCorporate   
                  94   44  bat          COCA:2008:ACAD\nBioscience   
                  79   29  bat     COCA:2014:ACAD\nJAnimalPlantSci   
                  100  50  bat         COCA:2003:ACAD\nNaturalHist   
                  66   16  bat             COCA:1992:MAG\nUSAToday   
                  86   36  bat         COCA:2004:ACAD\nNaturalHist   

                                                                                                                                                                                                                                                                                                                                                                                    sentence                                                                                                                                                                                                                                                                                                                                                               \
buchanan_cue_word                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
bat_animal        92                                                                                                        colonize the newly formed landscape. Animal life followed more slowly. Birds and arthropods were the first to arrive, and with the land to themselves, they evolved into thousands of species that occur only in these islands . No amphibian or terrestrial reptile, and only one terrestrial mammal, the hoary bat, completed the journey on its own. People were the late-comers: not untilsometime between 1,000 and 1,500 years ago did they first reach the islands. The original settlers were Polynesians, and with them began the rapid introduction, both intentional and inadvertent, of exotic fauna and flora. #   
                  57                                                                                                                                                                                                   of still water, and above all that a sharp ammonia stench. # Two hou

In [53]:
"""
no crane_(bird) data for mcrae
"""
print(len(eval_words))
eval_words = eval_words[eval_words['buchanan_cue_word'] != 'crane_animal']
print(len(eval_words))

# print(len(eval_words))
# eval_words = eval_words[eval_words['buchanan_cue_word'] != 'tank_army']
# print(len(eval_words))

794
780


In [54]:


"""
data is a list of tuples of (cue word, lemma, context)

for each model, we want to run an analysis of these
"""

    
def evaluate_in_context(model, df, bert, glove=False, model_name=None, dataset='mc_rae'):
    """
    df has columns
    sentence
    buchanan_cue_word
    mcrae_cue_word
    """
    n = 0
    
    cosines = []
    top_10_precs = []
    top_20_precs = []
    top_k_precs = []
    correlations = []
    rows = []

    num_top_10 = 0
    num_top_20 = 0
    num_total = 0

    for index, row in df.iterrows():
        if dataset == 'buchanan':
            cue_word = row.buchanan_cue_word  
        else:
            cue_word = row.mcrae_cue_word  

        context = row.sentence
        word = row.word
        
        n +=1


        prediction = model.predict_in_context(word, context, bert, glove=glove)
        #top_k =  model.predict_top_n_features_in_context(singular, context, k, bert)

        # get gold feature
        gold_vector = model.feature_norms.get_feature_vector(cue_word)
        gold_feats = model.feature_norms.get_features(cue_word)
        k = len(gold_feats)

        # get cosines
        cos = 1 - cosine(prediction, gold_vector)
        cosines.append(cos)

        # get MAP at 10
        top_10 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
        top_10_gold = model.feature_norms.top_n(cue_word, 10)
        
        num_in_top_10 = len(set(top_10).intersection(set(top_10_gold)))
        top_10_prec = num_in_top_10 / len(top_10_gold)
        top_10_precs.append(top_10_prec)

        # get MAP at 20
        top_20 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
        top_20_gold = model.feature_norms.top_n(cue_word, 20)

        num_in_top_20 = len(set(top_20).intersection(set(top_20_gold)))
        top_20_prec = num_in_top_20 / len(top_20_gold)
        top_20_precs.append(top_20_prec)

        # get MAP at k
        gold_len = len(gold_feats)
        top_k = model.predict_top_n_features_in_context(word, context, gold_len, vec=prediction, bert=bert)
        num_in_top_k = len(set(top_k).intersection(set(gold_feats)))
        top_k_prec = num_in_top_k / gold_len
        top_k_precs.append(top_k_prec)

        # get correlation
        corr, p = spearmanr(prediction, gold_vector)
        correlations.append(corr)

        #if ((i % 20 ==0) and debug=='info') or (debug=='true'):
#         if False:
#             print(word)
#             print("top ten predicted features: ", top_10)
#             print("top ten gold features: ", top_10_gold)
#             print("gold features: ", gold_feats)
#             print("top k predicted features: ", top_k)

#             print("cosine: %f" % cos)
#             print("correlation: %f" % corr)
#             print("top k prec: %f" % top_k_prec)
    
    
        row = (cue_word, context, word, top_10_prec, top_20_prec, top_k_prec, top_10, top_10_gold, top_k, gold_feats, cos)
        rows.append(row)
    
    cols = ['cue_word', 
               'context', 
               'lemma', 
               'top_10_prec', 
               'top_20_prec', 
               'top_k_prec',
               'top_10', 
               'top_10_gold', 
               'top_k', 
               'gold_feats', 
               'cos']
    
        
    avg_top_10_prec = np.average(top_10_precs)
    avg_top_20_prec = np.average(top_20_precs)
    avg_top_k_prec = np.average(top_k_precs)
    average_correlation = np.average(correlations)
    average_cosine = np.average(cosines)
        
        
    print("Average cosine between gold and predicted feature norms: %s" % average_cosine)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector: ", top_10_prec)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector: ", top_20_prec)
    print("Average % @k (derby metric)", top_k_prec)
    print("correlation between gold and predicted vectors: %s " % average_correlation)        
    print("total number of predictions: ", n)
    
    results_df = pd.DataFrame.from_records(rows, columns = cols)
    results_df['avg_top_10_prec'] = avg_top_10_prec
    results_df['avg_top_20_prec'] = avg_top_20_prec
    results_df['avg_top_k_prec'] = avg_top_k_prec
    results_df['avg_correlation'] = average_correlation
    results_df['avg_cosine'] = average_cosine
    
    # TODO make this possible!!!!
    # results_df['model'] = model.name ()
    results_df['model'] = model_name

    
    return results_df
    



# Evaluate McRae models

In [66]:
#dfs = []

for save_path in mcrae_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)

    df = evaluate_in_context(model, eval_words, bert, model_name=save_path, dataset='mc_rae')
    dfs.append(df)
    
# need to do this separately because we dont have nice self examining attributes for our model classes to tell when its 
# been trained on glove
for save_path in mcrae_glove_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path, dataset='mc_rae')
    dfs.append(df)

df = pd.concat(dfs)
df.to_csv('../results/homonymous_words_eval_mcrae_models_expanded_10_20_22.csv')

****************************************
*** Evaluating ../trained_models/main_d62d4_00011_11_clusters=5,embedding_type=bert,model=modabs,mu2=1e-07,mu3=0.001,mu4=20,nnk=6,train_data=mc_rae_real_2022-10-15_18-12-19 model ***
****************************************
fox_0
fox_0
owl_3
ant_3
owl_4
owl_3
squirrel_3
wasp_1
owl_4
owl_3
kite_3
owl_3
owl_4
owl_1
cat_4
frog_0
ant_3
owl_3
rat_3
owl_3
owl_4
cat_1
cat_4
owl_4
frog_0
cat_2
squirrel_1
owl_1
owl_3
goldfish_3
squirrel_1
ant_3
frog_0
cat_2
owl_4
owl_3
wasp_1
owl_3
owl_4
cat_2
grasshopper_1
owl_3
cat_1
cat_1
fox_0
frog_0
owl_3
cat_4
hatchet_3
frog_0
ball_4
ball_4
ball_3
skis_3
ball_4
ball_4
ball_4
ball_0
ball_4
ball_4
ball_0
ball_4
hatchet_0
ball_4
stick_0
ball_0
harpoon_3
ball_4
machete_2
ball_4
hatchet_0
sledgehammer_4
ball_4
ball_3
ball_4
dagger_0
ball_4
ball_4
ball_3
ball_4
ball_3
ball_4
ball_4
machete_1
ball_4
ball_0
ball_0
ball_4
ball_4
ball_4
ball_4
ball_3
ball_3
ball_0
ball_4
ball_4
sledgehammer_2
ball_3
machete_4
ball_3
plate_4


plate_0
screws_0
plate_0
surfboard_0
crowbar_0
plate_0
surfboard_0
surfboard_0
plate_0
plate_0
banner_0
spear_0
rope_0
spear_0
spear_0
rope_0
banner_0
rope_0
rope_0
tie_0
rope_0
rope_0
dagger_0
rope_0
bouquet_0
spear_0
bouquet_0
rope_0
rope_0
spear_0
slingshot_0
bouquet_0
rope_0
rope_0
rope_0
bouquet_0
pin_0
spear_0
earmuffs_0
rope_0
bouquet_0
bouquet_0
spear_0
rope_0
spear_0
earmuffs_0
bouquet_0
rope_0
rope_0
rope_0
rope_0
slingshot_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
anchor_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
crossbow_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
stick_0
spear_0
crossbow_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
spear_0
helmet_0
helmet_0
helmet_0
helmet_0
helmet_0
helmet_0
helmet_0
clamp_0
clamp_0
barrel_0
barrel_0
helmet_0
helmet_0
barrel_0
shield_0
clamp_0
helm

predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicti

predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting

In [67]:
df.head(100)

,cue_word,context,lemma,top_10_prec,top_20_prec,top_k_prec,top_10,top_10_gold,top_k,gold_feats,cos,avg_top_10_prec,avg_top_20_prec,avg_top_k_prec,avg_correlation,avg_cosine,model
0,bat_(animal),"colonize the newly formed landscape. Animal life followed more slowly. Birds and arthropods were the first to arrive, and with the land to themselves, they evolved into thousands of species that occur only in these islands . No amphibian or terrestrial reptile, and only one terrestrial mammal, the hoary bat, completed the journey on its own. People were the late-comers: not untilsometime between 1,000 and 1,500 years ago did they first reach the islands. The original settlers were Polynesians, and with them began the rapid introduction, both intentional and inadvertent, of exotic fauna and flora. #",bat,0.400,0.250,0.312,"[is_black, has_a_beak, a_bird, has_feathers, an_insect, beh_-_flies, has_wings, is_green, an_animal, is_large]","[an_animal, beh_-_sleeps_upside_down, is_blind, has_fur, lives_in_caves, is_black, beh_-_flies, beh_-_is_nocturnal, has_wings, associated_with_vampires]","[has_legs, a_tool, is_small, is_long, a_vegetable, beh_-_eats, is_black, has_a_beak, a_bird, has_feathers, an_insect, beh_-_flies, has_wings, is_green, an_animal, is_large]","[has_wings, beh_-_flies, beh_-_is_nocturnal, is_black, lives_in_caves, has_fur, an_animal, beh_-_sleeps_upside_down, is_blind, beh_-_uses_radar_to_navigate, beh_-_screeches, a_mammal, has_fangs, is_scary, is_small, associated_with_vampires]",0.436,0.247,0.225,0.242,0.06,0.273,../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
1,bat_(animal),"of still water, and above all that a sharp ammonia stench. # Two hours ago, a security guard named Halhuatl had walked along the reservoir, casting about in the dark with a bull's-eye lantern. Hearing a splash, he stepped forward. He saw nothing no night bird, no bat, no swimming coyote or bathing snake. He scanned the water with his lantern. Where the light touched, it left a rippling trail. # That's strange, Hal must have thought, before he died. # A chill wind blew over the water, producing no waves. Caleb",bat,0.300,0.188,0.250,"[has_a_beak, has_feathers, an_animal, is_large, an_insect, a_vegetable, a_bird, beh_-_flies, is_green, has_wings]","[an_animal, beh_-_sleeps_upside_down, is_blind, has_fur, lives_in_caves, is_black, beh_-_flies, beh_-_is_nocturnal, has_wings, associated_with_vampires]","[made_of_wood, has_legs, has_a_beak, is_long, is_brown, is_black, beh_-_eats, has_feathers, an_animal, is_large, an_insect, a_vegetable, a_bird, beh_-_flies, is_green, has_wings]","[has_wings, beh_-_flies, beh_-_is_nocturnal, is_black, lives_in_caves, has_fur, an_animal, beh_-_sleeps_upside_down, is_blind, beh_-_uses_radar_to_navigate, beh_-_screeches, a_mammal, has_fangs, is_scary, is_small, associated_with_vampires]",0.359,0.247,0.225,0.242,0.06,0.273,../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
2,bat_(animal),"to do with escaping roosts that had become heavily contaminated by blood-sucking ectoparasites. Ever since the fruit bats have been forced to take permanent residence in one location, the walls of that northern cave have been literally crawling with parasitic insects and their larvae. # So what explains the bald, toothless bat that was clamped onto my thumb in 1997? Before the onset of volcanic activitytwo years earlier, less than 1 percent of the fruit bats examined by biologists on Montserrat showed any sign of tooth wear or hair loss. The bats that did were elderly animals with other obvious signs of age",bat,0.400,0.250,0.312,"[has_a_beak, has_feathers, is_large, an_animal, has_wings, beh_-_flies, an_insect, a_bird, is_black, beh_-_eats]","[an_animal, beh_-_sleeps_upside_down, is_blind, has_fur, lives_in_caves, is_black, beh_-_flies, beh_-_is_nocturnal, has_wings, associated_with_vampires]","[has_a_shell, is_large, an_animal, has_legs, is_green, a

# Evaluate Buchanan models

In [68]:
dfs = []

for save_path in buchanan_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, model_name=save_path, dataset='buchanan')
    dfs.append(df)

for save_path in buchanan_glove_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path, dataset='buchanan')
    dfs.append(df)


df = pd.concat(dfs)
df.to_csv('../results/homonymous_words_eval_buchanan_models_expanded_10_20_22.csv')

****************************************
*** Evaluating ../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters model ***
****************************************
Average cosine between gold and predicted feature norms: 0.33524860722009764
average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector:  0.4
average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector:  0.4166666666666667
Average % @k (derby metric) 0.4166666666666667
correlation between gold and predicted vectors: 0.08428795902960032 
total number of predictions:  780
****************************************
*** Evaluating ../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters model ***
****************************************
Average cosine between gold and predicted feature norms: 0.31107224815121787
average Percentage (%) of gold gold-standard features retrieved in 

bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
bat_0
sledgehammer_0
bat_0
bat_0
bat_0
chalkboard_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
chalkboard_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board_0
board

predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicting features for  board
predicti

predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  hose
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting features for  mink
predicting

# Mcrae Results analysis

In [4]:
mcrae_res_df = pd.read_csv('../results/homonymous_words_eval_mcrae_models_expanded_10_20_22.csv')
mcrae_res_df = mcrae_res_df.rename(columns={'Unnamed: 0': 'token_index'})
mcrae_res_df.head(50)

,token_index,cue_word,context,lemma,top_10_prec,top_20_prec,top_k_prec,top_10,top_10_gold,top_k,gold_feats,cos,avg_top_10_prec,avg_top_20_prec,avg_top_k_prec,avg_correlation,avg_cosine,model
0,0,bat_(animal),colonize the newly formed landscape. Animal li...,bat,0.4,0.2500,0.3125,"['is_black', 'has_a_beak', 'a_bird', 'has_feat...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['has_legs', 'a_tool', 'is_small', 'is_long', ...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.435957,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
1,1,bat_(animal),"of still water, and above all that a sharp amm...",bat,0.3,0.1875,0.2500,"['has_a_beak', 'has_feathers', 'an_animal', 'i...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['made_of_wood', 'has_legs', 'has_a_beak', 'is...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.358547,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
2,2,bat_(animal),to do with escaping roosts that had become hea...,bat,0.4,0.2500,0.3125,"['has_a_beak', 'has_feathers', 'is_large', 'an...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['has_a_shell', 'is_large', 'an_animal', 'has_...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.450522,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
3,3,bat_(animal),on the basis of external morphology following ...,bat,0.3,0.3125,0.4375,"['a_mammal', 'has_a_tail', 'has_legs', 'has_fu...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['a_tool', 'has_wings', 'is_brown', 'beh_-_eat...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.354027,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
4,4,bat_(animal),vertebrates. Science 296: 69-72. # -- Conradt ...,bat,0.4,0.3125,0.3125,"['has_legs', 'is_black', 'a_bird', 'an_animal'...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['a_vegetable', 'is_long', 'has_a_beak', 'beh_...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.421172,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
5,5,bat_(animal),"for their survival (Mickleburgh et al., 1992)....",bat,0.4,0.2500,0.2500,"['is_black', 'beh_-_eats', 'has_wings', 'an_in...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['beh_-_stings', 'is_green', 'a_tool', 'beh_-_...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.435195,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
6,6,bat_(animal),"a lot worse than it is. Besides, I've got plen...",bat,0.3,0.1875,0.3125,"['made_of_metal', 'a_tool', 'an_animal', 'a_ve...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['has_a_beak', 'made_of_wood', 'is_black', 'ha...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.349060,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
7,7,bat_(animal),Burnbauer has forgotten me. Carlisle is in the...,bat,0.2,0.1250,0.2500,"['is_green', 'a_tool', 'has_a_handle', 'made_o...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['made_of_metal', 'has_a_beak', 'a_vegetable',...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.324538,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
8,8,bat_(animal),causes and consequences of sociality in animal...,bat,0.3,0.3125,0.3125,"['is_furry', 'an_animal', 'a_mammal', 'is_larg...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['beh_-_eats', 'a_carnivore', 'a_musical_instr...","['has_wings', 'beh_-_flies', 'beh_-_is_nocturn...",0.323878,0.24723,0.225372,0.241543,0.060346,0.272941,../trained_models/model.ffnn.mc_rae_real.allbu...
9,9,bat_(animal),volcano partially destroyed the Belham River a...,bat,0.4,0.3125,0.3750,"['has_feathers', 'has_legs', 'beh_-_eats', 'is...","['an_animal', 'beh_-_sleeps_upside_down', 'is_...","['is_long

In [6]:
def analyze_results(model_name, res_df):
    # get best and worst performing words on plsr 5k --- avg prec .5
    print("****************************************")
    print("*** Analyzing results for %s model ***" % model_name)
    print("****************************************")

    print("best 5 examples")
    print(res_df[res_df.model == model_name].sort_values(by=['top_k_prec'], ascending=False).head(5))

    print("worst 5 examples")
    print(res_df[res_df.model == model_name].sort_values(by=['top_k_prec'], ascending=True).head(5))

In [7]:
for model in mcrae_models:
    analyze_results(model, mcrae_res_df)
    


****************************************
*** Analyzing results for ../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
best 5 examples
     token_index          cue_word  \
746          746  tank_(container)   
733          733  tank_(container)   
757          757  tank_(container)   
777          777  tank_(container)   
760          760  tank_(container)   

                                               context lemma  top_10_prec  \
746  STEEL # Liquids that are used in pharmaceutica...  tank          0.6   
733  maintain cool storage temperatures in warmer w...  tank          0.6   
757  expansive forces within the concrete that resu...  tank          0.6   
777  of the hydrogen energy is required for the liq...  tank          0.6   
760  the permanent onboard tank. The containers wou...  tank          0.6   

     top_20_prec  top_k_prec  \
746          0.6         0.6   
733          0.

# Buchanan Results Analysis

In [21]:
buchanan_res_df = pd.read_csv('../results/homonymous_words_eval_buchanan_models_expanded_10_20_22.csv')
buchanan_res_df = buchanan_res_df.rename(columns={'Unnamed: 0': 'token_index'})
buchanan_res_df.head(5)


,token_index,cue_word,context,lemma,top_10_prec,top_20_prec,top_k_prec,top_10,top_10_gold,top_k,gold_feats,cos,avg_top_10_prec,avg_top_20_prec,avg_top_k_prec,avg_correlation,avg_cosine,model
0,0,bat_animal,"volcano partially destroyed the Belham River and obliterated a quirky thirteen-hole golf course that had meandered across the river's bottomlands. Although the episode was clearly a setback for the Montserratian golfing community, the flows were catastrophic to a unique ecosystem that was also the island's only known habitat for the fishing bat (Noctilio leporinus). # Fishing bats are large, yellow-orange, and ratherpungent creatures that can hawk large flying insects or snag small ocean fish from the surf. But they much prefer to take minnows from the surface of freshwater streams and ponds -- exactly what the course of the Belham",bat,0.3,0.389,0.389,"['insect', 'small', 'black', 'animal', 'fur', 'eat', 'wing', 'fly', 'mammal', 'leg']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['play', 'long', 'beak', 'tail', 'large', 'feather', 'four', 'leg', 'fur', 'small', 'black', 'animal', 'bird', 'eat', 'wing', 'fly', 'mammal', 'insect']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.390,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
1,1,bat_animal,"change the way they roll. Hey, you're a friend of Tony Zucco's, ai n't you? I hear he's been feeling the heat lately. Lucky five. Yeah, the Z-man's been lying low, all right. They say he's got bat problems. Big bat problems. I can't help wondering how he's keeping out of sight.You do an awful lot of wondering, for someone we ai n't ever seen before. What'd you say your name was? Smith. Smith. That would n't be Detective Smith, would it? Or",bat,0.1,0.167,0.278,"['hand', 'fly', 'leg', 'long', 'fur', 'object', 'wood', 'large', 'animal', 'human']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['mammal', 'intent', 'fur', 'cold', 'sport', 'protect', 'small', 'hand', 'four', 'fly', 'leg', 'long', 'act', 'object', 'wood', 'large', 'animal', 'human']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.236,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
2,2,bat_animal,"the baleful look. She sensed that I was the dragonslayer. She knew there was a real dragon out here. She'd met him. p204 "" I remember I ran away, escaped into woods just like these. Tall Carolina pines. Not much light getting through, eerie as a bat cave. I remember clearly when the house disappeared on me. I can'tremember too much else. I'm blocking it. I do n't even know how I got into the river. "" We were about two miles from where we'd left the car. Now we hiked north",bat,0.2,0.333,0.333,"['insect', 'fur', 'eat', 'wing', 'leg', 'animal', 'wood', 'black', 'small', 'fly']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['human', 'bird', 'four', 'cloth', 'long', 'cold', 'large', 'air', 'insect', 'fur', 'eat', 'wing', 'leg', 'animal', 'wood', 'black', 'small', 'fly']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.331,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
3,3,bat_animal,", in many temperate-zone species, the males disperse from their natal area and settle at places where they do not mate. Avoidance of local resource competition with females may be an explanation for male dispersal in some speci

In [ ]:
# which model does best?

In [13]:
# which words get highest precision? lowest?

print(mcrae_res_df.model.unique())

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)


['../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters'
 '../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters'
 '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300'
 '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300'
 '../trained_models/main_d62d4_00011_11_clusters=5,embedding_type=bert,model=modabs,mu2=1e-07,mu3=0.001,mu4=20,nnk=6,train_data=mc_rae_real_2022-10-15_18-12-18'
 '../trained_models/main_094eb_00082_82_clusters=1,embedding_type=bert,model=modabs,mu2=0.1,mu3=10.0,mu4=20,nnk=6,train_data=buchanan_2022-10-12_05-39-12'
 '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters'
 '../trained_models/model.ffnn.buchanan.allbuthomonyms.glove.50epochs.0.0dropout.lr1e-4.hsize300'
 '../trained_models/main_094eb_00035_35_clusters=1,embedding_type=glove,model=modabs,mu2=1e-07,mu3=1.0,mu4=5,nnk=6,train_data=buchanan_

In [15]:
mcrae_res_df.head(5)

,token_index,cue_word,context,lemma,top_10_prec,top_20_prec,top_k_prec,top_10,top_10_gold,top_k,gold_feats,cos,avg_top_10_prec,avg_top_20_prec,avg_top_k_prec,avg_correlation,avg_cosine,model
0,0,bat_animal,"volcano partially destroyed the Belham River and obliterated a quirky thirteen-hole golf course that had meandered across the river's bottomlands. Although the episode was clearly a setback for the Montserratian golfing community, the flows were catastrophic to a unique ecosystem that was also the island's only known habitat for the fishing bat (Noctilio leporinus). # Fishing bats are large, yellow-orange, and ratherpungent creatures that can hawk large flying insects or snag small ocean fish from the surf. But they much prefer to take minnows from the surface of freshwater streams and ponds -- exactly what the course of the Belham",bat,0.3,0.389,0.389,"['insect', 'small', 'black', 'animal', 'fur', 'eat', 'wing', 'fly', 'mammal', 'leg']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['play', 'long', 'beak', 'tail', 'large', 'feather', 'four', 'leg', 'fur', 'small', 'black', 'animal', 'bird', 'eat', 'wing', 'fly', 'mammal', 'insect']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.390,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
1,1,bat_animal,"change the way they roll. Hey, you're a friend of Tony Zucco's, ai n't you? I hear he's been feeling the heat lately. Lucky five. Yeah, the Z-man's been lying low, all right. They say he's got bat problems. Big bat problems. I can't help wondering how he's keeping out of sight.You do an awful lot of wondering, for someone we ai n't ever seen before. What'd you say your name was? Smith. Smith. That would n't be Detective Smith, would it? Or",bat,0.1,0.167,0.278,"['hand', 'fly', 'leg', 'long', 'fur', 'object', 'wood', 'large', 'animal', 'human']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['mammal', 'intent', 'fur', 'cold', 'sport', 'protect', 'small', 'hand', 'four', 'fly', 'leg', 'long', 'act', 'object', 'wood', 'large', 'animal', 'human']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.236,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
2,2,bat_animal,"the baleful look. She sensed that I was the dragonslayer. She knew there was a real dragon out here. She'd met him. p204 "" I remember I ran away, escaped into woods just like these. Tall Carolina pines. Not much light getting through, eerie as a bat cave. I remember clearly when the house disappeared on me. I can'tremember too much else. I'm blocking it. I do n't even know how I got into the river. "" We were about two miles from where we'd left the car. Now we hiked north",bat,0.2,0.333,0.333,"['insect', 'fur', 'eat', 'wing', 'leg', 'animal', 'wood', 'black', 'small', 'fly']","['vampire', 'scare', 'small', 'fang', 'mammal', 'screech', 'navigate', 'radar', 'animal', 'blind']","['human', 'bird', 'four', 'cloth', 'long', 'cold', 'large', 'air', 'insect', 'fur', 'eat', 'wing', 'leg', 'animal', 'wood', 'black', 'small', 'fly']","['animal', 'black', 'blind', 'cave', 'down', 'fang', 'fly', 'fur', 'mammal', 'navigate', 'nocturnal', 'radar', 'scare', 'screech', 'sleep', 'small', 'vampire', 'wing']",0.331,0.231,0.253,0.284,0.079,0.314,../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
3,3,bat_animal,", in many temperate-zone species, the males disperse from their natal area and settle at places where they do not mate. Avoidance of local resource competition with females may be an explanation for male dispersal in some speci

In [23]:
# buchanan_models = [
#     '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters',
#     '../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters',
#     '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
#     #'../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.2dropout.lr1e-4.hsize300',
#     '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
#     '../trained_models/main_d62d4_00011_11_clusters=5,embedding_type=bert,model=modabs,mu2=1e-07,mu3=0.001,mu4=20,nnk=6,train_data=mc_rae_real_2022-10-15_18-12-18', # mcrae is just a misnomer this was trained on buchanan, or else it wouldnt work
#     '../trained_models/main_094eb_00082_82_clusters=1,embedding_type=bert,model=modabs,mu2=0.1,mu3=10.0,mu4=20,nnk=6,train_data=buchanan_2022-10-12_05-39-12',
# ]


# buchanan_glove_models = [
#     '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters',
#     '../trained_models/model.ffnn.buchanan.allbuthomonyms.glove.50epochs.0.0dropout.lr1e-4.hsize300',
#     '../trained_models/main_094eb_00035_35_clusters=1,embedding_type=glove,model=modabs,mu2=1e-07,mu3=1.0,mu4=5,nnk=6,train_data=buchanan_2022-10-12_02-26-58'   
# ]




# get best and worst performing words on plsr 5k --- avg prec .5
print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters'
].sort_values(by=['top_k_prec'], ascending=True).head(5))

     token_index    cue_word    \
10        10      bat_baseball   
116      116              mink   
113      113              mink   
114      114              mink   
115      115              mink   

                                                                                                                                                                                                                                                                                                                                     context                                                                                                                                                                                                                                                                                                                                   \
10                                                                                                                                        

In [26]:

# get best and worst performing words on glove plsr
print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters'

].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters'

].sort_values(by=['top_k_prec'], ascending=True).head(10))

     token_index cue_word  \
898      118       mink     
899      119       mink     
890      110       mink     
891      111       mink     
892      112       mink     

                                                                                                                                                                                                                                                                                                               context                                                                                                                                                                                                                                                                                                             \
898                                                and does not imply endorsement by the U.S. Government. Footnote # 1 Present address: Madison Audubon Society, Madison, Wisconsin 53703 # 2 Corresponding author: 

In [27]:
#'../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300'
# get best and worst performing words on ffnn 5k 
print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=True).head(5))


     token_index    cue_word    \
279      19       bat_baseball   
270      10       bat_baseball   
315      55         bow_weapon   
319      59         bow_weapon   
317      57         bow_weapon   

                                                                                                                                                                                                                                                                                                                context                                                                                                                                                                                                                                                                                                               \
279                                                                                                   around in the outfield during batting practice and catch fly balls. I'd play 

In [28]:



# get best and worst performing words on ffnn 1k --- avg prec .5
print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(buchanan_res_df[buchanan_res_df.model == '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=True).head(5))

     token_index    cue_word    \
403      13       bat_baseball   
409      19       bat_baseball   
440      50         bow_weapon   
449      59         bow_weapon   
447      57         bow_weapon   

                                                                                                                                                                                                                                                                                                 context                                                                                                                                                                                                                                                                                                \
403                                  lines before games, signing autographs and tossing them balls. # That said, safety has to be paramount for all MLB teams.Given the pace of the game, in many cases, fans are

In [ ]:


# get best and worst performing words on glove ffnn
print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=True).head(10))

## Look at differential features for tank

In [ ]:
# Lets look side by side at the predictions for a few homonymous pairs, across all models.
# we'll want to look at the gold features, the features predicted, and the scores, for each of the models

# tank
# army = 54
# container = 51

# get gold feats for tank
# print(mcrae_res_df[mcrae_res_df.token_index in (54, 51)][:1][['lemma','context', 'gold_feats']])
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([54,51])][['lemma','context', 'gold_feats']][:2]



In [ ]:
# get predicted feats for each of the models
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([54,51])][['model','lemma','top_10_prec', 'top_10']].sort_values(by='model')


## Look at differentia features for pipe

In [ ]:
# Lets look side by side at the predictions for a few homonymous pairs, across all models.
# we'll want to look at the gold features, the features predicted, and the scores, for each of the models

# pipe
# smoke = 38
# plumbing = 41



In [ ]:
# get predicted feats for each of the models
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([38,41])][['model','lemma','top_10_prec', 'top_10']].sort_values(by='model')


# Correlate predicted features with wu-palmer similarity

use homonym token dataset
 lemma cue_word context 
 
add label row to df

Get predictions on 

In [ ]:
df = pd.read_csv('../data/bnc_contexts_for_mcrae_homonyms.csv')
df.head(5)

In [ ]:
# transform wordnet string into wordnet lemma

ws = []
wn_lemmas = []
for index, row in df.iterrows():
    lemma = re.findall(r"'(.*?)'", row.lemma)[0]
    lemma = wn.lemma(lemma)
    
    word = lemma.name()
    #print(word)
    ws.append(word)
    wn_lemmas.append(lemma)
    
df['label'] = ws
df['wn_lemma'] = wn_lemmas

df.head(5)


In [ ]:
def run_wu_palmer_analysis(df):
    """
    input is a dataframe with columns
        cue_word
        lemma
        context
        label
    """
    wup_sims = []
    cossine_sims = []

    for index, row in df.iterrows():
        word = row.label

        # select other examples of this token
        othertokens = df[df.label == word]
        # filter out the token itself
        othertokens = othertokens[othertokens.index != index]
        #print(othertokens)

        for index, otherword in othertokens.iterrows():
            # find the wordnet distance between these two wordnet senses
            synset1 = row.wn_lemma.synset()
            synset2 = otherword.wn_lemma.synset()

            wup_sim = synset1.wup_similarity(synset2)
            wup_sims.append(wup_sim)
            cossim = 1 - cosine(row.predictions, otherword.predictions)
            #cossim = 1 - cosine(row.single_prototype_model_preds, otherword.single_prototype_model_preds)
            cossine_sims.append(cossim)
            #print(synset1)
            #print(synset2)
            #print(wup_sim)
            #print(cossim)
    return (wup_sims, cossine_sims)

def plot_sims(wup_sims, cossine_sims):
    plt.scatter(wup_sims, cossine_sims)
    plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
    plt.xlabel("Wu and Palmer Similarity")
    plt.ylabel("Cosine Similarity")
    plt.show()

In [ ]:
"""
create a list of predicted feature vectors for each of the models being evaluated
and then stick them in the dataframe
"""

for save_path in mcrae_models:
    print("****************************************")
    print("*** doing wu palmer correlation for %s ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    
    predictions = []
    for index, row in df.iterrows():
            singular = row.wn_lemma.name()
            plural = pluralize(singular)

            try:
                predicted_vector = model.predict_in_context(singular, row.context, bert)
            except:
                predicted_vector = model.predict_in_context(plural, row.context, bert)


            predictions.append(predicted_vector)

    df['predictions'] = predictions
    
    wup_sims, cossine_sims = run_wu_palmer_analysis(df)
    plot_sims(wup_sims, cossine_sims)

    corr, p = pearsonr(wup_sims, cossine_sims)
    print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

    corr, p = spearmanr(wup_sims, cossine_sims)
    print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

# Run the analysis for the non-contextual model (trained on GloVe)

Here we expect to see no correlation, because the model should be making the same prediction for every token of a word form, irrespective of the surrounding context

In [ ]:
"""
create a list of predicted feature vectors for each of the models being evaluated
and then stick them in the dataframe
"""

for save_path in mcrae_glove_models:
    print("****************************************")
    print("*** doing wu palmer correlation for %s ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    
    predictions = []
    for index, row in df.iterrows():
            singular = row.wn_lemma.name()
            plural = pluralize(singular)

            try:
                predicted_vector = model.predict_in_context(singular, row.context, bert, glove=True)
            except:
                predicted_vector = model.predict_in_context(plural, row.context, bert, glove=True)


            predictions.append(predicted_vector)

    df['predictions'] = predictions
    
    wup_sims, cossine_sims = run_wu_palmer_analysis(df)
    plot_sims(wup_sims, cossine_sims)

    corr, p = pearsonr(wup_sims, cossine_sims)
    print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

    corr, p = spearmanr(wup_sims, cossine_sims)
    print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

In [ ]:
embeddings_list = []
word_indexer = Indexer()
with open("../data/glove.6B/glove.6B.300d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_list.append([vector])

        #print(embeddings_dict)
        #raise Exception("hfelfnl")
        word_indexer.add_and_get_index(word)

embs = MultiProtoTypeEmbeddings(word_indexer, np.array(embeddings_list), 0, 1)
